In [67]:
import pandas as pd
import numpy as np
import re
from pathlib import Path

In [68]:
CSV_PATH = r"C:\Users\asila\Downloads\strsearch_takehome\data\Blue Ridge GA - Market Eval - FINAL - Base_Table.csv"
OUT_PATH = r"C:\Users\asila\Downloads\strsearch_takehome\data\clean\blue_ridge_clean.csv"

In [69]:
df = pd.read_csv(CSV_PATH, low_memory=False)

In [70]:
print(df.shape)

(906, 101)


In [71]:
def pick_first(row, cols):
    for c in cols:
        if c in row and pd.notna(row[c]):
            return row[c]
    return np.nan

def to_int_series(s):
    # handles "21.0" and empty strings safely
    s = pd.to_numeric(s.astype(str).str.strip().replace({"": np.nan, "nan": np.nan, "None": np.nan}), errors="coerce")
    return s.round(0).astype("Int64")

def to_float_series(s):
    return pd.to_numeric(s.astype(str).str.strip().replace({"": np.nan, "nan": np.nan, "None": np.nan}), errors="coerce").astype(float)

def to_bool_series(s):
    # normalizes True/False, Yes/No, 1/0, Y/N (string or numeric)
    def norm(x):
        if pd.isna(x): return pd.NA
        if isinstance(x, str):
            v = x.strip().lower()
            if v in {"true","t","yes","y","1"}: return True
            if v in {"false","f","no","n","0"}: return False
            return pd.NA
        if isinstance(x, (int, float, np.integer, np.floating)):
            if x == 1 or x == 1.0: return True
            if x == 0 or x == 0.0: return False
        return pd.NA
    return s.map(norm).astype("boolean")

In [72]:
id_col = "Property ID"

title_cols = ["TITLE", "title", "Listing Name", "name"]
lat_cols = ["LATITUDE", "latitude"]
lon_cols = ["LONGITUDE", "longitude"]
bed_cols = ["BEDROOMS", "Bedrooms"]
acc_cols = ["ACCOMMODATES", "personCapacity", "personCapacity "]
bath_cols = ["BATHROOMS", "baths", "Bathrooms"]
zip_cols = ["ZIPCODE", "zipcode"]

# boolean columns: keep HAS_*, SYSTEM_*, Has_*, plus some flags
bool_cols = [c for c in df.columns if c.startswith("HAS_") or c.startswith("SYSTEM_") or c.startswith("Has_")]
for extra in ["INSTANT_BOOK", "SUPERHOST", "is_super_host", "is_guest_favorite"]:
    if extra in df.columns:
        bool_cols.append(extra)
if "SYSTEM_GYM" in bool_cols:
    bool_cols.remove("SYSTEM_GYM")
bool_cols = sorted(set(bool_cols))

In [73]:
clean = pd.DataFrame()
clean["property_id"] = df[id_col].astype(str).str.strip()

clean["title"] = df.apply(lambda r: pick_first(r, title_cols), axis=1)
clean["title"] = clean["title"].astype(str).replace("nan", np.nan)

host_raw = df.get("Property Manager/ Host ID")
if host_raw is None:
    clean["host_id"] = np.nan
else:
    clean["host_id"] = (
        host_raw.astype(str).str.strip()
        .replace({"nan": np.nan, "None": np.nan, "": np.nan})
    )

clean["host_id"] = df["Property Manager/ Host ID"] if "Property Manager/ Host ID" in df.columns else np.nan


clean["airbnb_listing_url"] = df.get("Airbnb Listing URL")
clean["vrbo_listing_url"] = df.get("Vrbo Listing URL")
clean["airbnb_host_url"] = df.get("Airbnb Host URL")
clean["url"] = df.get("url")

clean["city"] = df.get("CITY_NAME")
clean["state"] = df.get("STATE_NAME")
clean["zipcode"] = df.apply(lambda r: pick_first(r, zip_cols), axis=1)

clean["latitude"] = to_float_series(df.apply(lambda r: pick_first(r, lat_cols), axis=1))
clean["longitude"] = to_float_series(df.apply(lambda r: pick_first(r, lon_cols), axis=1))
clean["url"] = df.get("url")



In [74]:
# property basics
clean["bedrooms"] = to_int_series(df.apply(lambda r: pick_first(r, bed_cols), axis=1))
clean["accommodates"] = to_int_series(df.apply(lambda r: pick_first(r, acc_cols), axis=1))
clean["bathrooms"] = to_float_series(df.apply(lambda r: pick_first(r, bath_cols), axis=1))
clean["minimum_stay"] = to_int_series(df.get("MINIMUM_STAY", pd.Series([np.nan]*len(df))))

# performance
clean["revenue"] = to_float_series(df.get("Revenue", pd.Series([np.nan]*len(df))))
clean["revenue_potential"] = to_float_series(df.get("Revenue Potential", pd.Series([np.nan]*len(df))))
clean["adr"] = to_float_series(df.get("ADR", pd.Series([np.nan]*len(df))))
clean["occupancy"] = to_float_series(df.get("Occupancy", pd.Series([np.nan]*len(df))))
clean["cleaning_fee"] = to_float_series(df.get("Cleaning Fee", pd.Series([np.nan]*len(df))))
clean["available_nights"] = to_int_series(df.get("Available Nights", pd.Series([np.nan]*len(df))))
pt = df.get("PRICE_TIER")
if pt is None:
    clean["price_tier"] = np.nan
else:
    pt_str = pt.astype(str).str.strip()

    clean["price_tier"] = (
        pt_str
        # remove leading number(s)
        .str.replace(r"^\s*\d+\s*", "", regex=True)
        # remove leading punctuation like "." "-" "_" and extra spaces
        .str.replace(r"^[\.\-\_\s]+", "", regex=True)
        # collapse multiple spaces
        .str.replace(r"\s+", " ", regex=True)
        # fix empty/nan-ish
        .replace({"nan": np.nan, "None": np.nan, "": np.nan})
        # optional: standardize casing
        .str.upper()
    )


clean["total_reviews"] = to_int_series(df.get("total_reviews", pd.Series([np.nan]*len(df))))
clean["property_rating"] = to_float_series(df.get("Property Rating", pd.Series([np.nan]*len(df))))
clean["stars"] = to_float_series(df.get("stars", pd.Series([np.nan]*len(df))))

# text
clean["description"] = df.get("description")
clean["amenities_text"] = df.get("amenities")

In [75]:
for c in bool_cols:
    clean[c.lower()] = to_bool_series(df[c])

In [76]:
print("Clean shape:", clean.shape)
print("Missing bedrooms:", clean["bedrooms"].isna().mean())
print("Missing revenue:", clean["revenue"].isna().mean())

Clean shape: (906, 59)
Missing bedrooms: 0.0
Missing revenue: 0.0


In [77]:
print(clean.head())

                property_id  \
0   abnb_843427409328707195   
1  abnb_1072358047706308102   
2   abnb_843427434784984731   
3             abnb_47115055   
4             abnb_17864330   

                                               title    host_id  \
0   Lake Front Luxury Pool Hot tub Movie Theater Gym  504577995   
1  Views, Pool, Sport Court, Gym, Sauna, Petting Zoo  190951764   
2       Luxury Riverfront Pool Theater Games Hot Tub  504577995   
3  Toccoa Lodge | Toccoa River Frontage | 10,000 ...   47350784   
4  Luxe Mountain Resort - Heated Pool & 12ft Hot Tub  122265519   

                                 airbnb_listing_url  \
0   https://www.airbnb.com/rooms/843427409328707195   
1  https://www.airbnb.com/rooms/1072358047706308102   
2   https://www.airbnb.com/rooms/843427434784984731   
3             https://www.airbnb.com/rooms/47115055   
4             https://www.airbnb.com/rooms/17864330   

               vrbo_listing_url                              airbnb_host_url  \

In [78]:
Path(OUT_PATH).parent.mkdir(parents=True, exist_ok=True)
clean.to_csv(OUT_PATH, index=False)

In [79]:
print("Clean columns:", len(clean.columns))
for c in clean.columns:
    print("-", c)


Clean columns: 59
- property_id
- title
- host_id
- airbnb_listing_url
- vrbo_listing_url
- airbnb_host_url
- url
- city
- state
- zipcode
- latitude
- longitude
- bedrooms
- accommodates
- bathrooms
- minimum_stay
- revenue
- revenue_potential
- adr
- occupancy
- cleaning_fee
- available_nights
- price_tier
- total_reviews
- property_rating
- stars
- description
- amenities_text
- has_aircon
- has_gym
- has_hottub
- has_kitchen
- has_parking
- has_pets_allowed
- has_pool
- has_beach_access
- has_lake_access
- has_outdoor_dining_area
- has_outdoor_furniture
- has_waterfront
- instant_book
- superhost
- system_arcade_machine
- system_bowling
- system_chess
- system_crib
- system_firepit
- system_golf
- system_grill
- system_jacuzzi
- system_movie
- system_pack_n_play
- system_play_slide
- system_pool
- system_pool_table
- system_view_mountain
- system_view_ocean
- is_guest_favorite
- is_super_host


In [80]:
print("\nNumeric columns:")
print(clean.select_dtypes(include=["number", "Int64", "float64"]).columns.tolist())

print("\nBoolean columns:")
print(clean.select_dtypes(include=["boolean", "bool"]).columns.tolist())

print("\nText/object columns:")
print(clean.select_dtypes(include=["object"]).columns.tolist())



Numeric columns:
['zipcode', 'latitude', 'longitude', 'bedrooms', 'accommodates', 'bathrooms', 'minimum_stay', 'revenue', 'revenue_potential', 'adr', 'occupancy', 'cleaning_fee', 'available_nights', 'total_reviews', 'property_rating', 'stars']

Boolean columns:
['has_aircon', 'has_gym', 'has_hottub', 'has_kitchen', 'has_parking', 'has_pets_allowed', 'has_pool', 'has_beach_access', 'has_lake_access', 'has_outdoor_dining_area', 'has_outdoor_furniture', 'has_waterfront', 'instant_book', 'superhost', 'system_arcade_machine', 'system_bowling', 'system_chess', 'system_crib', 'system_firepit', 'system_golf', 'system_grill', 'system_jacuzzi', 'system_movie', 'system_pack_n_play', 'system_play_slide', 'system_pool', 'system_pool_table', 'system_view_mountain', 'system_view_ocean', 'is_guest_favorite', 'is_super_host']

Text/object columns:
['property_id', 'title', 'host_id', 'airbnb_listing_url', 'vrbo_listing_url', 'airbnb_host_url', 'url', 'city', 'state', 'price_tier', 'description', 'ameni